In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib nbagg

import time
import numpy as np
import matplotlib.pyplot as plt

from robmob.icp import icp
from robmob.robot import Robot
from robmob.sensors import HokuyoSensor

In [ ]:
robot = Robot('192.168.0.108')
robot.connect()

In [ ]:
hokuyo = HokuyoSensor()
robot.add_sensor(hokuyo)

In [ ]:
def hokuyo_to_3d_homogeneous_matrix(hokuyo_data):
    ranges = np.array(hokuyo_data['ranges'])
    thetas = np.arange(hokuyo_data['angle_min'], hokuyo_data['angle_max']+hokuyo_data['angle_increment'], hokuyo_data['angle_increment'])
    xs = ranges * np.cos(thetas)
    ys = ranges * np.sin(thetas)
    
    pts = np.vstack((xs, ys, np.zeros(xs.shape), np.ones(xs.shape))).T
    
    # Remove the points that are at the origin.
    pts = pts[np.nonzero(np.logical_or(pts[:,0] != 0., pts[:,1] != 0.))]
    
    return pts

In [ ]:
def plot_point_clouds(pcls):
    STYLES = ['r+', 'bx', 'g*']
    
    for i, pcl in enumerate(pcls):
        plt.plot(pcl[:,0], pcl[:,1], STYLES[i])
        plt.plot([0.0], [0.0], 'ko', markersize=10)
        
    plt.axis('equal')
    
    plt.show()

In [ ]:
reading1 = hokuyo.peek_data()
pcl1 = hokuyo_to_3d_homogeneous_matrix(reading1)

In [ ]:
reading2 = hokuyo.peek_data()
pcl2 = hokuyo_to_3d_homogeneous_matrix(reading2)

In [ ]:
plot_point_clouds([pcl1, pcl2])

In [ ]:
H, errs = icp(pcl1, pcl2)
print('La distance moyenne entre les points associés est de {}'.format(np.average(errs)))

In [ ]:
H

In [ ]:
moved_pcl = H.dot(pcl1.T)
moved_pcl = moved_pcl.T
moved_pcl

In [ ]:
plot_point_clouds([pcl1, pcl2, moved_pcl])

In [ ]:
pcl2